In [6]:
#Code to download all needed libraries - needs to be run every time

from __future__ import division #used in Python2.x so the division btw integers result in float, default in python3
from pyomo.environ import * #library for modeling and optimisation
import argparse
from pyomo.opt import SolverStatus, TerminationCondition #provide status of solvers
import pandas as pd #data manipulation library
import numpy as np 
import pickle
import random
import sqlite3
import copy
import random
import csv

import warnings
warnings.filterwarnings('ignore')

INTEREST = 0.03 #interest rate

scen = 500 #number of scenarios for stochastic optimization


path = "C:/VoI/"  #specify the path to the working folder

In [66]:
#THIS CODE GENERATED SCENARIOS AND CREATES NEW DATAFRAME WITH ASSIGNED SCENARIOS 
#ONLY RUN THIS WHEN GENERATIN SCENARIOS FOR THE FIRST TIME

data_opt =  pd.read_csv(path+"direct_10_22.csv")  #Specify the input file
id = set(data_opt.id)

num_iterations = 6 #change according to number of the simulations for each stand in the input file

for k in range(0,scen):
    for i in list(id):
        if i == list(id)[0]:
            if k == 0:
                data_scen = data_opt[(data_opt['id']==i) & (data_opt['iteration']==random.choice(range(1,num_iterations)))]
                data_scen['SCENARIO'] = k
            else:
                data_scen_temp = data_opt[(data_opt['id']==i) & (data_opt['iteration']==random.choice(range(1,num_iterations)))]
                data_scen_temp['SCENARIO'] = k
                data_scen = pd.concat([data_scen,data_scen_temp])
        else:
            data_scen_temp = data_opt[(data_opt['id']==i) & (data_opt['iteration']==random.choice(range(1,num_iterations)))]
            data_scen_temp['SCENARIO'] = k
            data_scen = pd.concat([data_scen,data_scen_temp])

data_scen

data_scen.to_csv(path+"Input/scenario_tables/data_scen_direct_10_22_500.csv")  #Saves the new dataframe with scenarios, specify the path

In [7]:
#The code with the main model. Enough to run it once, after that shorter version could be used to change targets or other parameters

#This part of the code creates the optimization model

class optimization:
    def __init__(self):
        
        data_opt =  pd.read_csv(path+"Input/scenario_tables/data_scen_direct_10_22_500.csv") #Specify the path to input file with all scenarios (generated in previous step)
        all_data = data_opt
        all_data = all_data.set_index(['id','branch','year','SCENARIO']) #reassigns the Df with new multi-level index
        all_data = all_data.fillna(0) #fills any missing values with 0
        all_data['year'] = all_data.index.get_level_values(2)
        
        self.data_opt=all_data #store the fetched or preprocessed data for use within the class
        self.combinations = 1
        self.all_data = self.data_opt
        self.Index_values = self.all_data.drop(['year'], axis=1).reset_index().set_index(['id','branch']).index.unique() #we drop year, because we dont make decision for each year
        self.area = self.all_data.loc[slice(None),0,1,self.all_data.index.get_level_values(3).min()]['area']
        self.all_data = self.all_data.fillna(0)

        self.createModel()

    def createModel(self):
        # Declare sets - These used to recongnize the number of stands, regimes and number of periods in the analysis.
        # Define sets, variables, and constraints for the optimization problem
        
        self.model1 = ConcreteModel() #defining the model
        
        self.model1.stands = Set(initialize = list(set(self.all_data.index.get_level_values(0)))) #initialized with unique values from the first level of the index created earlier
        self.model1.year = Set(initialize = list(set(self.all_data.index.get_level_values(2)))) #initialized with unique values from the third level of the index created earlier
        self.model1.SCENARIO = Set(initialize = list(set(self.all_data.index.get_level_values(3))))        
        self.model1.regimes = Set(initialize = list(set(self.all_data.index.get_level_values(1))))
        self.model1.scen_index = Set(initialize= [i for i in range(0,self.combinations)])
        self.model1.Index_values = self.Index_values

        # Indexes (stand, regime)-- excludes those combinations that have no regimes simulated -- because some regimes are not simulated for some stands

        def index_rule(model1):
            index = []
            for (s,r) in model1.Index_values: 
                index.append((s,r))
            return index
        self.model1.index1 = Set(dimen=2, initialize=index_rule)

        #Decision variable
        self.model1.X1 = Var(self.model1.index1, within=NonNegativeReals, bounds=(0,1), initialize=1)  

        self.all_data['year'] = self.all_data.index.get_level_values(2)

        #Objective and constraint don't need to be adjusted here, leave as it is, used to create the optimization model in the code. 
        #objective function:
        def outcome_rule(model1):
            return sum((self.all_data.Harvested_V.loc[(s,r,k,it)]*self.all_data.area.loc[(s,r,k,it)]* self.model1.X1[(s,r)])/((1+INTEREST)**(2.5+self.all_data.year[(s,r,k,it)]))  for (s,r) in self.model1.index1 for k in self.model1.year for it in self.model1.SCENARIO)
        self.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)

        #Constraint:
        def regime_rule(model1, s):
            row_sum = sum(model1.X1[(s,r)] for r in [x[1] for x in model1.index1 if x[0] == s])
            return row_sum == 1
        self.model1.regime_limit = Constraint(self.model1.stands, rule=regime_rule)

    def solve(self):
        # Specify the solver and solve the model
        opt = SolverFactory('cplex') #Here we use CPLEX solver, could be adjusted to use open-sources ones (for example, cbc)
        self.results = opt.solve(self.model1,tee=False) #We solve a problem, but do not show the solver output

# Create an optimization object        
t1 = optimization()
t2 = copy.deepcopy(t1)

# Modify the optimization model to focus on. 

try:
    t2.model1.del_component(t2.model1.NPV_INV)
except:
    print("NONE")

#A function that evaluates NPV (already discounted in Gaya simulated output)  
t2.model1.NPV= Var(within=NonNegativeReals) 
def NPV_INVENTORY(model1):
    row_sum = sum(t2.all_data.NPV.loc[(s,r,10,it)]*t2.all_data.area.loc[(s,r,10,it)]* t2.model1.X1[(s,r)]  for (s,r) in t2.model1.index1 for it in t2.model1.SCENARIO)/scen
    return t2.model1.NPV ==row_sum
t2.model1.NPV_INV= Constraint(rule=NPV_INVENTORY)  

#A function that evaluates the combined HSI per stand

t2.all_data['CHSI'] = 1- ((1-t2.all_data.LSWP)*(1-t2.all_data.TTWP)*(1-t2.all_data.LTT)*(1-t2.all_data.CAP)*(1-t2.all_data.HAZ))

try:
    t2.model1.del_component(t2.model1.COMB_HSI)
except:
    print("NONE")

try:
    t2.model1.del_component(t2.model1.LAND_HSI)
except:
    print("NONE")

try:
    t2.model1.del_component(t2.model1.HSI)
except:
    print("NONE")

try:
    t2.model1.del_component(t2.model1.Landscape_HSI)
except:
    print("NONE")

t2.model1.HSI=Var(t2.model1.stands, t2.model1.year, within=NonNegativeReals)
def COMBINED_HSI(model1, s, k):
    birds = (sum(t2.model1.X1[(s,r)] * t2.all_data.CHSI.loc[(s,r,k,it)] for r in [x[1] for x in t2.model1.index1 if x[0] == s] for it in t2.model1.SCENARIO)) / scen
    return t2.model1.HSI[(s,k)] == birds
t2.model1.COMB_HSI = Constraint(t2.model1.stands,t2.model1.year, rule=COMBINED_HSI)

#A function that evaluates the combined HSI for the landscape
t2.model1.Landscape_HSI=Var(t2.model1.year, within=NonNegativeReals)
def Landscape_HSI(model1, k):
    Land_HSI = sum(t2.model1.HSI[(s,k)]*t2.area[s] for s in t2.model1.stands)
    return t2.model1.Landscape_HSI[k] == Land_HSI
t2.model1.LAND_HSI = Constraint(t2.model1.year, rule=Landscape_HSI)

#A function that computes the total value as sum per periods
t2.model1.Landscape_HSI_Tot = Var(within=NonNegativeReals)
def HSI_Tot(model1):
    row_sum = sum(t2.model1.Landscape_HSI[k] for k in t2.model1.year)
    return t2.model1.Landscape_HSI_Tot == row_sum
t2.model1.LAND_HSI_TOTAL = Constraint(rule=HSI_Tot)

#code for computing CVaR

#Downside CVaR
t2.model1.CVAR_down = Var(t2.model1.year,within=Reals, initialize=1)
t2.model1.VAR_down = Var(t2.model1.year,within=Reals, initialize=1)
t2.model1.posVAR_down = Var(t2.model1.year,t2.model1.SCENARIO, within=NonNegativeReals, initialize=1)
t2.model1.negVAR_down = Var(t2.model1.year,t2.model1.SCENARIO, within=NonNegativeReals, initialize=1)
t2.model1.mod_L_plus_down = Var(t2.model1.year,t2.model1.SCENARIO, within=NonNegativeReals, initialize=1)
t2.model1.mod_L_neg_down = Var(t2.model1.year,t2.model1.SCENARIO, within=NonNegativeReals, initialize=1)
t2.model1.alpha_risk_down = Param(default=0.80, mutable=True) #Specify here the confidence interval
t2.model1.target_down = Param(default=600000, mutable=True) #Specify here the desired minimum periodic income
t2.model1.CVAR_down_Tot = Var(within=Reals)


#Equations to computre downside CVaR
def CVAR_constraint_down(model1,k):
    CVAR_down = t2.model1.VAR_down[k] + (1/((1-t2.model1.alpha_risk_down)*scen))*sum(t2.model1.posVAR_down[k,scenario] for scenario in t2.model1.SCENARIO)
    return t2.model1.CVAR_down[k] == CVAR_down
t2.model1.CVAR_constraint_down = Constraint(t2.model1.year,rule=CVAR_constraint_down)

def MOD_L_P_constraint_down(model1,k,it):
    VAL_down = t2.model1.mod_L_plus_down[k,it] -t2.model1.VAR_down[k]+t2.model1.negVAR_down[k,it]-t2.model1.posVAR_down[k,it]
    return VAL_down == 0
t2.model1.MOD_L_P_down = Constraint(t2.model1.year,t2.model1.SCENARIO,rule=MOD_L_P_constraint_down)

def MOD_TARGET_constraint_down(model1,k,it):
    row_sum = sum(((t2.all_data.INC.loc[(s,r,k,it)])*t2.all_data.area.loc[(s,r,k,it)]* t2.model1.X1[(s,r)]) for (s,r) in t2.model1.index1)
    VAL = t2.model1.target_down-row_sum - t2.model1.mod_L_plus_down[k,it]+t2.model1.mod_L_neg_down[k,it]
    return VAL == 0
t2.model1.MOD_Target_down = Constraint(t2.model1.year,t2.model1.SCENARIO,rule=MOD_TARGET_constraint_down)

def CVAR_down_constraint(model1):
    row_sum = sum(t2.model1.CVAR_down[k] for k in t2.model1.year)
    return t2.model1.CVAR_down_Tot == row_sum 
t2.model1.CVAR_tot_down = Constraint(rule=CVAR_down_constraint)


#Upside CVaR
t2.model1.CVAR_up = Var(t2.model1.year,within=Reals, initialize=1)
t2.model1.VAR_up = Var(t2.model1.year,within=Reals, initialize=1)
t2.model1.posVAR_up = Var(t2.model1.year,t2.model1.SCENARIO, within=NonNegativeReals, initialize=1)
t2.model1.negVAR_up = Var(t2.model1.year,t2.model1.SCENARIO, within=NonNegativeReals, initialize=1)
t2.model1.mod_L_plus_up = Var(t2.model1.year,t2.model1.SCENARIO, within=NonNegativeReals, initialize=1)
t2.model1.mod_L_neg_up = Var(t2.model1.year,t2.model1.SCENARIO, within=NonNegativeReals, initialize=1)
t2.model1.alpha_risk_up = Param(default=0.80, mutable=True) #Specify here the confidence interval
t2.model1.target_up = Param(default=800000, mutable=True) #Specify here the desired maximum periodic income
t2.model1.CVAR_up_Tot = Var(within=Reals)

#Equations to computre upside CVaR
def CVAR_constraint_up(model1,k):
    CVAR_up = t2.model1.VAR_up[k] + (1/((1-t2.model1.alpha_risk_up)*scen))*sum(t2.model1.negVAR_up[k,scenario] for scenario in t2.model1.SCENARIO)
    return t2.model1.CVAR_up[k] == CVAR_up
t2.model1.CVAR_constraint_up = Constraint(t2.model1.year,rule=CVAR_constraint_up)

def MOD_L_P_constraint_up(model1,k,it):
    VAL_up = t2.model1.mod_L_neg_up[k,it] -t2.model1.VAR_up[k]-t2.model1.negVAR_up[k,it]+t2.model1.posVAR_up[k,it]
    return VAL_up == 0
t2.model1.MOD_L_P_up = Constraint(t2.model1.year,t2.model1.SCENARIO,rule=MOD_L_P_constraint_up)

def MOD_TARGET_constraint_up(model1,k,it):
    row_sum = sum(((t2.all_data.INC.loc[(s,r,k,it)])*t2.all_data.area.loc[(s,r,k,it)]* t2.model1.X1[(s,r)]) for (s,r) in t2.model1.index1)
    VAL = t2.model1.target_up - row_sum - t2.model1.mod_L_plus_up[k,it]+t2.model1.mod_L_neg_up[k,it]
    return VAL == 0
t2.model1.MOD_Target_up = Constraint(t2.model1.year,t2.model1.SCENARIO,rule=MOD_TARGET_constraint_up)

def CVAR_up_constraint(model1):
    row_sum = sum(t2.model1.CVAR_up[k] for k in t2.model1.year)
    return t2.model1.CVAR_up_Tot == row_sum 
t2.model1.CVAR_tot_up = Constraint(rule=CVAR_up_constraint)

#Equations to compute standing end volume 

t2.model1.EndVolume = Var(within=NonNegativeReals)
def End_volume_constraint(model1):
    row_sum = sum(t2.all_data.V_end.loc[(s,r,10,it)]*t2.all_data.area.loc[(s,r,10,it)]* t2.model1.X1[(s,r)]  for (s,r) in t2.model1.index1 for it in t2.model1.SCENARIO)/scen
    return t2.model1.EndVolume == row_sum
t2.model1.EndInv = Constraint(rule=End_volume_constraint)

#minimum and maximum values and target parameters. If knowm - adjust here, if unknown yet - will be computed later

t2.model1.NPV_min = Param(default=93064.4153858999, mutable=True)
t2.model1.NPV_max = Param(default=1412635.71273736, mutable=True)
t2.model1.NPV_target = Param(default=1500000, mutable=True)

t2.model1.HSI_min = Param(default=163.953968012528, mutable=True)
t2.model1.HSI_max = Param(default=239.51424384946, mutable=True)
t2.model1.HSI_target = Param(default=400, mutable=True)

t2.model1.CVAR_down_min = Param(default=126344.885279602, mutable=True)
t2.model1.CVAR_down_max = Param(default=6061218.19076652, mutable=True)
t2.model1.CVAR_down_target = Param(default=0, mutable=True)

t2.model1.CVAR_up_min = Param(default=0.00, mutable=True)
t2.model1.CVAR_up_max = Param(default=5324657.29371336, mutable=True)
t2.model1.CVAR_up_target = Param(default=0, mutable=True)

t2.model1.EndVol_min = Param(default=11110.158276522, mutable=True)
t2.model1.EndVol_max = Param(default=26633.5645745358, mutable=True)
t2.model1.EndVol_target = Param(default=0, mutable=True)



try:
    t2.model1.del_component(t2.model1.OBJ)
except:
    print("NONE")

#Objective function -- currently minimizing the downside CVaR   
def outcome_rule(model1):
    return - t2.model1.CVAR_down_Tot 
t2.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)

# Solve the modified optimization model
t2.solve()

#Function to extract decision variables from the optimized model
def GET_DECISION_DATA():
        st = []
        reg = []
        vals = []
        for (s,r) in t2.model1.index1:
            st = st+[s]
            reg = reg+[r] 
            vals = vals+[t2.model1.X1[(s,r)].value]
        data = {"id":st,"branch":reg,"value":vals}
        df= pd.DataFrame(data)
        df = df.set_index(['id','branch'])
        return df

# Extract decision data and merge with original dataset    
dec  = GET_DECISION_DATA()
merged_df = dec.merge(t2.all_data, left_index=True, right_index=True, how='left')



NONE
NONE
NONE
NONE
NONE
NONE


In [11]:
#This code creates a payoff table - can be used after the previous code is executed


def RE_TRY(t2,VAL):
    t2.model1.del_component(t2.model1.OBJ)
    def outcome_rule1(model1):
        if VAL == "NPV":
            return t2.model1.NPV  - t2.model1.CVAR_down_Tot*0.00001 - t2.model1.CVAR_up_Tot*0.00001  #CVaR is included to ensure its proper calculation. Multiplied by a small number to ensure it does not influence the optimal solution 
        elif VAL == "END":
            return t2.model1.EndVolume - t2.model1.CVAR_down_Tot*0.00001 - t2.model1.CVAR_up_Tot*0.00001
        elif VAL == "CVAR_up":
            return - t2.model1.CVAR_up_Tot - t2.model1.CVAR_down_Tot*0.0000001
        elif VAL == "CVAR_down":
            return - t2.model1.CVAR_down_Tot - t2.model1.CVAR_up_Tot*0.0000001
        elif VAL == "HSI":
            return t2.model1.Landscape_HSI_Tot - t2.model1.CVAR_down_Tot*0.000001 - t2.model1.CVAR_up_Tot*0.000001
    t2.model1.OBJ = Objective(rule=outcome_rule1, sense=maximize)
    # Solve the modified optimization model
    t2.solve()
    t2a ={"NPV":t2.model1.NPV.value,"EndVolume": t2.model1.EndVolume.value,"CVAR_up_Tot": t2.model1.CVAR_up_Tot.value,"CVAR_down_Tot": t2.model1.CVAR_down_Tot.value,"Landscape_HSI_Tot": t2.model1.Landscape_HSI_Tot.value}
    return t2a
t2a = RE_TRY(t2,"NPV")
t2b = RE_TRY(t2,"END")
t2c = RE_TRY(t2,"CVAR_up")
t2d = RE_TRY(t2,"CVAR_down")
t2e = RE_TRY(t2,"HSI")
 

data = [t2a, t2b,t2c,t2d,t2e]
 
# Creating a DataFrame
df = pd.DataFrame(data)
(df.T).to_csv(path+"Outputs/Payoff_tables/payoff_direct.csv")  #Define the path to save the payoff table

In [44]:
#This code will extract the minimum and maximum values from the payoff table

import csv

def get_max_and_min_values_in_row(csv_file, row_number):
    with open(csv_file, newline='') as file:
        reader = csv.reader(file)
        # Skip the header row
        next(reader)
        for _ in range(row_number - 1):
            try:
                row = next(reader)
            except StopIteration:
                return None, None, None, None
        
        variable_name = row[0].strip()  
        values = []
        for value in row[1:]:
            try:
                numeric_value = float(value)
                values.append(numeric_value)
            except ValueError:
                pass  
        if values:
            max_value = max(values)
            min_value = min(values)
        else:
            max_value = None
            min_value = None
        return variable_name + "_Max", max_value, variable_name + "_Min", min_value

#The path to payoff table
csv_file = path+"Outputs/Payoff_tables/payoff_direct.csv"

for row_number in range(2, 7):  # Identify the rown with values, skipping the header
    max_var_name, max_value, min_var_name, min_value = get_max_and_min_values_in_row(csv_file, row_number)
    if max_var_name is not None:
        locals()[max_var_name] = max_value
    if min_var_name is not None:
        locals()[min_var_name] = min_value


#Store the extracted minimum and maximum values as parameters for the optimization model
t2.model1.NPV_min = NPV_Min
t2.model1.NPV_max = NPV_Max

t2.model1.HSI_min = Landscape_HSI_Tot_Min
t2.model1.HSI_max = Landscape_HSI_Tot_Max

t2.model1.CVAR_down_min = CVAR_down_Tot_Min
t2.model1.CVAR_down_max = CVAR_down_Tot_Max

t2.model1.CVAR_up_min = CVAR_up_Tot_Min
t2.model1.CVAR_up_max = CVAR_up_Tot_Max

t2.model1.EndVol_min = EndVolume_Min
t2.model1.EndVol_max = EndVolume_Max


4281021.491273181 2517359.7537621805
33395.8714681528 3952.862299524629
4158671.894452451 0.0
5974176.839872 0.0
216.89822420154042 74.13831636700425


In [57]:
#This code defines the achievement-scalarizing objective function and runs the model for final outputs 

t2.model1.D = Var(within=Reals)

def NPV_D_constraint(model1):
    row_sum = ((t2.model1.NPV - t2.model1.NPV_target)/(t2.model1.NPV_max-t2.model1.NPV_min))
    return t2.model1.D <= row_sum 
t2.model1.NPV_D = Constraint(rule=NPV_D_constraint)
 
def End_V_D_constraint(model1):
    row_sum = ((t2.model1.EndVolume - t2.model1.EndVol_target)/(t2.model1.EndVol_max - t2.model1.EndVol_min))
    return t2.model1.D <= row_sum 
t2.model1.END_V_D = Constraint(rule=End_V_D_constraint)
 
def HSI_D_constraint(model1):
    row_sum = ((t2.model1.Landscape_HSI_Tot - t2.model1.HSI_target)/(t2.model1.HSI_max - t2.model1.HSI_min))
    return t2.model1.D <= row_sum 
t2.model1.HSI_D = Constraint(rule=HSI_D_constraint)

def CVAR_down_D_constraint(model1):
    row_sum = ((t2.model1.CVAR_down_target-t2.model1.CVAR_down_Tot)/(t2.model1.CVAR_down_max - t2.model1.CVAR_down_min))
    return t2.model1.D <= row_sum 
t2.model1.CVAR_down_D = Constraint(rule=CVAR_down_D_constraint)

def CVAR_up_D_constraint(model1):
    row_sum = ((t2.model1.CVAR_up_target-t2.model1.CVAR_up_Tot)/(t2.model1.CVAR_up_max - t2.model1.CVAR_up_min))
    return t2.model1.D <= row_sum 
t2.model1.CVAR_up_D = Constraint(rule=CVAR_up_D_constraint)
 
#Decision-maker's targets defined here
t2.model1.NPV_target = 3500000
t2.model1.CVAR_down_target = 5500000
t2.model1.CVAR_up_target = 3500000
t2.model1.HSI_target = 190
t2.model1.EndVol_target = 26000
 
#Objective function    

def outcome_rule(model1):
    return ((t2.model1.D + 0.001*(((t2.model1.NPV)/(t2.model1.NPV_max-t2.model1.NPV_min)) + 
            ((t2.model1.EndVolume)/(t2.model1.EndVol_max - t2.model1.EndVol_min)) + 
            ((t2.model1.Landscape_HSI_Tot)/(t2.model1.HSI_max - t2.model1.HSI_min)) -
            ((t2.model1.CVAR_down_Tot)/(t2.model1.CVAR_down_max - t2.model1.CVAR_down_min)) -
            ((t2.model1.CVAR_up_Tot)/(t2.model1.CVAR_up_max - t2.model1.CVAR_up_min))
            ) )*10000)
t2.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)


t2.solve()

#Function to extract decision variables from the optimized model
def GET_DECISION_DATA():
        st = []
        reg = []
        vals = []
        for (s,r) in t2.model1.index1:
            st = st+[s]
            reg = reg+[r] 
            vals = vals+[t2.model1.X1[(s,r)].value]
        data = {"id":st,"branch":reg,"value":vals}
        df= pd.DataFrame(data)
        df = df.set_index(['id','branch'])
        return df

# Extract decision data and merge with original dataset    
dec  = GET_DECISION_DATA()
merged_df = dec.merge(t2.all_data, left_index=True, right_index=True, how='left')

# Save results to a CSV file
merged_df.to_csv(path+"Outputs/direct_target1.csv")  #Specify the path to save the output

'pyomo.core.base.var.ScalarVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.AbstractScalarVar'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of

In [46]:
#Print values for main criteria

print("NPV:")
print(t2.model1.NPV.value)

print("CVaR_down:")
print(t2.model1.CVAR_down_Tot.value)

print("CVaR_up:")
print(t2.model1.CVAR_up_Tot.value)

print("Landscape HSI total:")
print(t2.model1.Landscape_HSI_Tot.value)

print("Expected end inventory volume")
print(t2.model1.EndVolume.value)


NPV:
2620801.1669585253
CVaR_down:
28517464.812615003
CVaR_up:
0.0
Landscape HSI total:
216.89822420154047
Expected end inventory volume
32588.119057917083


In [62]:
#Compute the distances to the targets

print((t2.model1.NPV.value - t2.model1.NPV_target.value)/(t2.model1.NPV_max.value-t2.model1.NPV_min.value)) 
print((t2.model1.CVAR_down_target.value - t2.model1.CVAR_down_Tot.value)/(t2.model1.CVAR_down_max.value - t2.model1.CVAR_down_min.value))
print((t2.model1.CVAR_up_target.value -t2.model1.CVAR_up_Tot.value)/(t2.model1.CVAR_up_max.value - t2.model1.CVAR_up_min.value)) 
print((t2.model1.Landscape_HSI_Tot.value - t2.model1.HSI_target.value)/(t2.model1.HSI_max.value - t2.model1.HSI_min.value)) 
print((t2.model1.EndVolume.value - t2.model1.EndVol_target.value)/(t2.model1.EndVol_max.value - t2.model1.EndVol_min.value))  

-0.042518728856289226
0.1382971029794552
0.5161621802973769
-0.042518728856289094
-0.0425187288562892
